# Load in clip


In [ ]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-L/14", device=device)


# Load in multi ling clip and ocr



In [ ]:
from multilingual_clip import pt_multilingual_clip
import transformers
# ff75d234-434a-4430-8eee-48a6bea519f8
# 0d7fc7c9-f599-11e9-b6ed-001999480be2
# ffad79cd-e301-11e6-b8fb-001999480be2
ocr_path = "CLIP\\filtered_texts\\ffad79cd-e301-11e6-b8fb-001999480be2.xml"
model_name = 'M-CLIP/XLM-Roberta-Large-Vit-L-14'

from pero_ocr.core.layout import PageLayout
page_layout = PageLayout(file=ocr_path)
texts = []
for region in page_layout.regions:
    if hasattr(region, 'polygon'):
        for line in region.lines:
            texts.append(line.transcription)
print(texts)

# # Load Model & Tokenizer
mult_model = pt_multilingual_clip.MultilingualCLIP.from_pretrained(model_name)
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

text_embeddings = mult_model.forward(texts, tokenizer)
print(text_embeddings.shape)


# Load in image

In [ ]:
image_path = "CLIP\\cropped_images\\ffad79cd-e301-11e6-b8fb-001999480be2.jpg"
image = Image.open(image_path).convert("RGB")
image_input = preprocess(image).unsqueeze(0).to(device)
with torch.no_grad():
    image_features = model.encode_image(image_input)
print(image_features.shape)

In [ ]:
import torch.nn.functional as F

image_features_norm = F.normalize(image_features, dim=-1)
text_features_norm = F.normalize(text_embeddings, dim=-1)
similarities = torch.matmul(image_features_norm, text_features_norm.T)[0].cpu().tolist()
for i,similarity in enumerate(similarities):
    print(f"{similarity}: {texts[i]}")
# print(similarities)